In [1]:
import pandas as pd
import numpy as np
import chromadb
from chromadb import Client
from chromadb import Settings
from sentence_transformers import SentenceTransformer

c:\Users\mados\anaconda3\envs\BSAN-6088\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# import the champions data
champions_df = pd.read_csv('champions.csv')

In [3]:
champions_df

,champion_name,lore_link,response_code,biography
0,aatrox,https://universe.leagueoflegends.com/en_US/sto...,200.0,"Whether mistaken for a demon or god, many tale..."
1,ahri,https://universe.leagueoflegends.com/en_US/sto...,200.0,"For most of her life, Ahri's origins were a my..."
2,akali,https://universe.leagueoflegends.com/en_US/sto...,200.0,"Ionia has always been a land of wild magic, it..."
3,akshan,https://universe.leagueoflegends.com/en_US/sto...,200.0,Dashing through the shadows of eastern Shurima...
4,alistar,https://universe.leagueoflegends.com/en_US/sto...,200.0,"Many civilizations have resisted Noxus, but no..."
...,...,...,...,...
163,zeri,https://universe.leagueoflegends.com/en_US/sto...,200.0,"Raised in a large working-class family, Zeri g..."
164,ziggs,https://universe.leagueoflegends.com/en_US/sto...,200.0,"Ziggs was born with a talent for tinkering, bu..."
165,zilean,https://universe.leagueoflegends.com/en_US/sto...,200.0,"Icathia, most desolate and cursed of lands, wa..."
166,zoe,https://universe.leagueoflegends.com/en_US/sto...,200.0,"As befits her Targonian Aspectâs nature, Zoe..."


In [4]:
texts = champions_df['biography'].tolist()
names = champions_df['champion_name'].tolist()

In [5]:
# Initialize the ChromaDB client
client = chromadb.PersistentClient(path='chromachampions.db')
collection = client.create_collection('chromachampions')

In [6]:
# Initialize the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts)

In [7]:
print(champions_df['champion_name'])

0       aatrox
1         ahri
2        akali
3       akshan
4      alistar
        ...   
163       zeri
164      ziggs
165     zilean
166        zoe
167       zyra
Name: champion_name, Length: 168, dtype: object


In [8]:
# Insert the data into the collection
# Prepare the data to be added
ids = [str(i) for i in range(len(texts))]
metadatas = [{"name": name, "biography": text} for name, text in zip(names, texts)]

In [9]:
# Add the data into the collection
collection.add(
    ids=ids,
    embeddings=embeddings,
    metadatas=metadatas
)

In [10]:
# query the collection
document = collection.query(query_texts=["Aatrox"],
                 n_results=1)

In [11]:
# Parse the result for metadata biography
print(document)

{'ids': [['0']], 'distances': [[1.3354880809783936]], 'metadatas': [[{'biography': 'Whether mistaken for a demon or god, many tales have been told of the Darkin Blade... but few know his real name, or the story of his fall.In ancient times, long before desert sands swallowed the empire, a mighty champion of Shurima was brought before the Sun Disc to become the avatar for a now forgotten celestial ideal. Remade as one of the Ascended, his wings were the golden light of dawn, and his armor sparkled like a constellation of hope from beyond the great veil.Aatrox was his name. He was at the vanguard of every noble conflict. So true and just was his conduct that other god-warriors would always gather at his side, and ten thousand mortals of Shurima marched behind him. When Setaka, the Ascended warrior-queen, called for his help against the rebellion of Icathia, Aatrox answered without hesitation.But no one predicted the extent of the horrors that the rebels would unleashâ\x80\x94the Void qui

In [22]:
document["metadatas"][0][0]["biography"]

'Whether mistaken for a demon or god, many tales have been told of the Darkin Blade... but few know his real name, or the story of his fall.In ancient times, long before desert sands swallowed the empire, a mighty champion of Shurima was brought before the Sun Disc to become the avatar for a now forgotten celestial ideal. Remade as one of the Ascended, his wings were the golden light of dawn, and his armor sparkled like a constellation of hope from beyond the great veil.Aatrox was his name. He was at the vanguard of every noble conflict. So true and just was his conduct that other god-warriors would always gather at his side, and ten thousand mortals of Shurima marched behind him. When Setaka, the Ascended warrior-queen, called for his help against the rebellion of Icathia, Aatrox answered without hesitation.But no one predicted the extent of the horrors that the rebels would unleashâ\x80\x94the Void quickly overwhelmed its Icathian masters, and began the grinding annihilation of all l

In [12]:
# # print the documents in the query and store it as text
# for document in document["metadatas"]:
#     context = document["metadatas"][0][0]
#     print(context["access_role"])

# # print the first item in the dictionary

NameError: name 'documents' is not defined